In [114]:
import requests
from datetime import datetime
import telebot
from telebot import types
import sqlite3
import random

import numpy as np
import pandas as pd
import psutil
from IPython.display import Image

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

import kaleido
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import collections
import plotly.express as px

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Черновик

In [ ]:
#https://api.hh.ru/
#GET/vacancies
#'https://api.hh.ru/vacancies?text=java'
#HH-User-Agent
#

Поиск вакансий по запросу. Описание метода тут https://github.com/hhru/api/blob/master/docs/vacancies.md

In [168]:
# r=requests.get('https://api.hh.ru/vacancies?text=python&per_page=100', headers={"User-Agent":"HH-User-Agent"})

поиск по вакансиям, похожим на вакансию

In [169]:
#GET/vacancies/{vacancy_id}/similar_vacancies

получение инфы о вакансии по id


In [170]:
#GET/vacancies/{vacancy_id}

поиск по компаниям. Описание тут https://github.com/hhru/api/blob/master/docs/employers.md#search

In [171]:
#GET/employers   можно указать критерии поиска компаний

#GET/employers/{employer_id}     возвращает данные о компании со ссылкой на выдачу вакансий этой компании

## Приступаем к получению данных

Сначала посмотрим, как вообще выглядит джейсонина

In [2]:
#получение списка зарплат
"""a = []
for i in r['items']:
    try:
        a.append((i['salary']['from'], i['salary']['to'], i['salary']['currency']))
    except:
        pass""";

In [3]:
search = 'python'
r = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100', headers={"User-Agent":"HH-User-Agent"}).json()

In [4]:
vac = r['items'][0]
vac

{'id': '30005963',
 'premium': False,
 'name': 'Python developer',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 4000, 'to': None, 'currency': 'USD', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': None,
  'street': None,
  'building': None,
  'description': None,
  'lat': None,
  'lng': None,
  'raw': None,
  'metro': {'station_name': 'Бауманская',
   'line_name': 'Арбатско-Покровская',
   'station_id': '3.17',
   'line_id': '3',
   'lat': 55.772405,
   'lng': 37.67904},
  'metro_stations': [{'station_name': 'Бауманская',
    'line_name': 'Арбатско-Покровская',
    'station_id': '3.17',
    'line_id': '3',
    'lat': 55.772405,
    'lng': 37.67904},
   {'station_name': 'Комсомольская',
    'line_name': 'Кольцевая',
    'station_id': '5.55',
    'line_id': '5',
    'lat': 55.775672,
    'lng': 37.654772}],
  'id': '1602971'},


In [5]:
vac['name']

'Python developer'

In [6]:
vac['salary']

{'from': 4000, 'to': None, 'currency': 'USD', 'gross': False}

In [7]:
vac['area']

{'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}

In [8]:
vac['address']

{'city': None,
 'street': None,
 'building': None,
 'description': None,
 'lat': None,
 'lng': None,
 'raw': None,
 'metro': {'station_name': 'Бауманская',
  'line_name': 'Арбатско-Покровская',
  'station_id': '3.17',
  'line_id': '3',
  'lat': 55.772405,
  'lng': 37.67904},
 'metro_stations': [{'station_name': 'Бауманская',
   'line_name': 'Арбатско-Покровская',
   'station_id': '3.17',
   'line_id': '3',
   'lat': 55.772405,
   'lng': 37.67904},
  {'station_name': 'Комсомольская',
   'line_name': 'Кольцевая',
   'station_id': '5.55',
   'line_id': '5',
   'lat': 55.775672,
   'lng': 37.654772}],
 'id': '1602971'}

In [9]:
vac['alternate_url'] #ссылка на вакансию на сайте

'https://hh.ru/vacancy/30005963'

In [10]:
vac['employer']

{'id': '899854',
 'name': 'NetworkOptix',
 'url': 'https://api.hh.ru/employers/899854',
 'alternate_url': 'https://hh.ru/employer/899854',
 'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/765059.png',
  '90': 'https://hhcdn.ru/employer-logo/3501193.png',
  '240': 'https://hhcdn.ru/employer-logo/3501194.png'},
 'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=899854',
 'trusted': True}

In [11]:
vac['url'] #ссылка на получение подробной инфы о вакансии через апи

'https://api.hh.ru/vacancies/30005963?host=hh.ru'

### Пишем функции для получения инфы

In [16]:
"""params = {
        'text': 'инженер', # Текст фильтра. В имени должно быть слово "Аналитик
        'per_page': 61, # Кол-во вакансий на 1 странице
    }"""

In [12]:
#вспомогательная функция для форматирования инфы о зарплате

def get_salary(sal):
    if sal['currency'] == None:
        sal['currency'] = ''
    if sal['from'] == None and sal['to'] == None:
        t = 'зарплата не указана'
        return t
    if sal['from'] == None:
        t = f'зарплата до {sal["to"]} {sal["currency"]}'
        return t
    if sal['to'] == None:
        t = f'зарплата от {sal["from"]} {sal["currency"]}'
        return t
    else:
        t = f'зарплата от {sal["from"]} {sal["currency"]} до {sal["to"]} {sal["currency"]}'
        return t

In [19]:
# основная функция получения инфы о вакансиях. Принимает запрос, по которому искать вакансию (любой текст)
# и номер вакансии в выдаче (ноль из ста по умолчанию). Если номер указан больше, чем есть в выдаче, то берет последнюю
# Возвращает инфу о названии вакансии, зарплате, регионе (но не городе подробно. Это можно будет прикрутить, если нужно)
# а также ссылку на вакансию с сайта хэ хэ

def get_vacancy_main(search, number = 0):
    data = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100', 
                        headers={"User-Agent":"HH-User-Agent"}).json()
    
    if len(data['items']) == 0:
        return 'вакансий по этому запросу не найдено'
    
    #получаем нужную вакансию из списка
    data = data['items'][min(number, len(data['items']) - 1)]
    
    name = data['name']
    
    try:
        salary = get_salary(data['salary'])
    except:
        salary = 'зарплата не указана'
    
    try:
        area = data['area']['name']
    except:
        area = 'регион не указан'
    
    if data['alternate_url'] != None:
        url = data['alternate_url']
    else:
        url = 'ссылка на вакансию отсутствует'
    
    
    
    return (name, salary, area, url)

парс зарплат

## Визуализация

функция для получения списка зарплат по запросу. Выдает два списка: минимльных и максимальных зарплат 

In [246]:
def all_salaries(search):
    
    #заводим пустой список для всех зарплат и делаем тестовый запрос, чтобы узнать, сколько страниц в выдаче
    salaries = []
    test = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100',
                        headers={"User-Agent":"HH-User-Agent"}).json()
    pages = test['pages']
    
    #проходимся по всем страницам в выдаче и склеиваем инфу о зарплатах. Парсится максимум 10 страниц, так как иначе очень долго
    for i in range(min(pages, 10)):
        data = requests.get(f'https://api.hh.ru/vacancies?text={search}&per_page=100&page={i}', 
                            headers={"User-Agent":"HH-User-Agent"}).json()
        for vac in data['items']:
            salaries.append(vac['salary'])
            
    #достаем из списка всех зарплат верхние и нижние границы    
    salaries_from = []
    salaries_to = []

    for salary in salaries:
        if salary != None:
            if salary['currency'] in ['RUR', 'RUB']:
                if salary['from'] != None:
                    salaries_from.append(salary['from'])
                if salary['to'] != None:
                    salaries_to.append(salary['to'])   
    
    return (salaries_from, salaries_to)

функция для построения гистограмм

In [242]:
def histogram(data):
    fig = px.histogram(data, nbins=30)
    
    fig.update_layout(
        title_text='распределение зарплат', # title of plot
        xaxis_title_text='зарплата', # xaxis label
        yaxis_title_text='количество вакансий', # yaxis label
        bargap=0.01, # gap between bars of adjacent location coordinates
        bargroupgap=0.01 # gap between bars of the same location coordinates
    )

    fig.update_layout(autosize=True,
                          width=800,
                          height=500,)
    
    return fig


# БОТ

In [191]:
#import plotly.io as pio

In [192]:
#pio.kaleido.scope.plotlyjs = r'D:\PythonModules\plotly\package_data\plotly.min.js'

In [243]:
token = "1866111504:AAFOl1jrVlrK00yoJQkDRGeFtNB0-iXh5u4" #ААААА спалил токен
bot = telebot.TeleBot(token)

In [244]:
@bot.message_handler(content_types=['text'])
def responser(message, text='а там армяне в нарды играют'):

    data = all_salaries(message.text)
    try: 
        result_photo = histogram(data[0]).to_image(format="png", engine="kaleido")
        bot.send_photo(message.chat.id, result_photo)
    except:
        bot.send_message(message.chat.id, 'введите другой запрос')


In [245]:
bot.polling(none_stop=True)